In [ ]:
# NOTEBOOK 2 - AGENTE (1→5 + conversacional + tools separadas) - ACTUALIZADO
# Incluye: carrito en memoria (dict), remover items, ver carrito, finalizar_compra (compra realizada + local + productos),
# y flujo "dónde comprar" (stock -> contacto).

import os
import json
import uuid
import logging
import mlflow
from typing import Any, Dict, List, Optional, Tuple

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv


# -----------------------
# LOGGING
# -----------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    datefmt="%H:%M:%S",
)
logger = logging.getLogger("NOTEBOOK_AGENTE")


# =========================================================
# 1. CONFIG
# =========================================================
if not load_dotenv(dotenv_path="../.env"):
    load_dotenv(dotenv_path=".env")

NEO4J_URI = os.getenv("NEO4J_URI")
if NEO4J_URI and "neo4j" in NEO4J_URI and "localhost" not in NEO4J_URI:
    logger.warning("⚠️ Detectado entorno local: Cambiando host 'neo4j' por 'localhost'")
    NEO4J_URI = NEO4J_URI.replace("neo4j", "localhost")

NEO4J_AUTH = (os.getenv("NEO4J_USER", "neo4j"), os.getenv("NEO4J_PASSWORD"))
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or ""

tracking_uri = os.getenv("MLFLOW_TRACKING_URI_LOCAL") or "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("Agente_Conversacional_Planner")

embedder = SentenceTransformer("all-MiniLM-L6-v2")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


# =========================================================
# 2. ESTADO LOCAL (Notebook) - carrito + selección
# =========================================================
STATE: Dict[str, Any] = {
    "stage": "explore",              # explore | decide | buy | contact | done
    "selected_product_id": None,
    "selected_store": None,
    "last_candidates": [],           # [{id,nombre,precio}]
    "cart_items": [],                # [{id,nombre,precio,qty}]
}


def reset_state():
    global STATE
    STATE = {
        "stage": "explore",
        "selected_product_id": None,
        "selected_store": None,
        "last_candidates": [],
        "cart_items": [],
    }


def _safe_int(x, default=1) -> int:
    try:
        return int(x)
    except Exception:
        return default


def normalize_ordinal_to_index(text: str) -> Optional[int]:
    t = (text or "").lower().strip()
    mapping = {
        "1": 0, "primera": 0, "primer": 0, "primero": 0, "la primera": 0, "el primero": 0,
        "2": 1, "segunda": 1, "segundo": 1, "la segunda": 1, "el segundo": 1,
        "3": 2, "tercera": 2, "tercero": 2, "la tercera": 2, "el tercero": 2,
        "4": 3, "cuarta": 3, "cuarto": 3, "la cuarta": 3, "el cuarto": 3,
        "5": 4, "quinta": 4, "quinto": 4, "la quinta": 4, "el quinto": 4,
    }
    for k, v in mapping.items():
        if k in t:
            return v
    return None


def cart_add_item(item: Dict[str, Any], qty: int = 1) -> None:
    qty = max(1, _safe_int(qty, 1))
    pid = item.get("id")
    if not pid:
        return

    for c in STATE["cart_items"]:
        if c.get("id") == pid:
            c["qty"] = _safe_int(c.get("qty", 1), 1) + qty
            return

    STATE["cart_items"].append({
        "id": pid,
        "nombre": item.get("nombre"),
        "precio": item.get("precio"),
        "qty": qty,
    })


def cart_clear() -> None:
    STATE["cart_items"] = []


def cart_to_text() -> str:
    cart = STATE.get("cart_items", []) or []
    if not cart:
        return "🛒 Carrito vacío."
    total = 0
    txt = "🛒 Carrito:\n"
    for i, c in enumerate(cart, start=1):
        precio = c.get("precio") or 0
        qty = _safe_int(c.get("qty", 1), 1)
        subtotal = precio * qty
        total += subtotal
        txt += f"{i}) {c.get('nombre')} [{c.get('id')}] x{qty} = ${subtotal}\n"
    txt += f"Total estimado: ${total}\n"
    return txt


def parse_cart_indexes(text: str) -> List[int]:
    raw = (text or "").lower()
    raw = raw.replace("items", "").replace("item", "")
    raw = raw.replace(" y ", ",").replace(";", ",")
    parts = [p.strip() for p in raw.split(",") if p.strip()]
    idxs = []
    for p in parts:
        try:
            n = int(p)
            if n > 0:
                idxs.append(n - 1)
        except Exception:
            continue
    return idxs


def cart_remove_by_indexes(indexes_0based: List[int]) -> List[Dict[str, Any]]:
    cart = STATE.get("cart_items", []) or []
    if not cart:
        return []
    idx_set = set(i for i in indexes_0based if isinstance(i, int))
    removed = []
    new_cart = []
    for i, item in enumerate(cart):
        if i in idx_set:
            removed.append(item)
        else:
            new_cart.append(item)
    STATE["cart_items"] = new_cart
    return removed


def cart_remove_by_name_or_id(item_ref: str) -> bool:
    cart = STATE.get("cart_items", []) or []
    ref = (item_ref or "").strip().lower()
    if not ref:
        return False

    new_cart = []
    removed = False
    for c in cart:
        cid = str(c.get("id", "")).lower()
        nombre = str(c.get("nombre", "")).lower()
        if ref == cid or ref in nombre:
            removed = True
            continue
        new_cart.append(c)

    STATE["cart_items"] = new_cart
    return removed


# =========================================================
# 3. TOOLS (ahora incluyen carrito + finalizar compra)
# =========================================================
@tool
def buscar_productos(query: str) -> str:
    """Explorar catálogo: devuelve productos y precios (NO stock). Guarda candidatos para usar ordinales."""
    logger.info(f"🛠️ [TOOL] buscar_productos | query='{query}'")
    v = embedder.encode(query).tolist()

    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    cypher = """
    CALL db.index.vector.queryNodes('productos_embeddings', 5, $vector)
    YIELD node AS p, score
    WHERE score > 0.5

    OPTIONAL MATCH (p)-[:COMPATIBLE_CON]->(acc:Producto)
    OPTIONAL MATCH (p)-[:TIENE_CORRECCION]->(c:Aprendizaje)

    RETURN
      p.id AS id,
      p.nombre AS nombre,
      p.precio AS precio,
      p.descripcion AS desc,
      collect(DISTINCT acc.nombre) AS accesorios,
      collect(DISTINCT c.nota) AS correcciones
    """
    try:
        with driver.session() as session:
            rows = [dict(r) for r in session.run(cypher, vector=v)]

        if not rows:
            STATE["stage"] = "explore"
            STATE["last_candidates"] = []
            STATE["selected_product_id"] = None
            return "No se encontraron productos similares."

        candidates = [{"id": r["id"], "nombre": r["nombre"], "precio": r["precio"]} for r in rows]
        STATE["last_candidates"] = candidates
        STATE["selected_product_id"] = candidates[0]["id"]
        STATE["stage"] = "decide"

        txt = "Opciones encontradas (sin ver stock todavía):\n"
        for i, r in enumerate(rows, start=1):
            txt += f"{i}) [{r['id']}] {r['nombre']} (${r['precio']})\n"
            txt += f"   Desc: {r['desc']}\n"
            if r["correcciones"]:
                txt += f"   🚨 Correcciones aprendidas: {r['correcciones']}\n"
            if r["accesorios"]:
                txt += f"   💡 Accesorios: {', '.join(r['accesorios'])}\n"
            txt += "\n"
        return txt
    finally:
        driver.close()


@tool
def agregar_al_carrito(producto_ref: str, qty: int = 1) -> str:
    """
    Agrega producto al carrito.
    producto_ref puede ser:
    - ordinal: "la segunda" / "2"
    - id: "L1"
    - nombre aproximado: "MacBook Air"
    """
    logger.info(f"🛠️ [TOOL] agregar_al_carrito | ref='{producto_ref}' qty={qty}")
    ref = (producto_ref or "").strip()
    if not ref:
        return "¿Qué producto agrego? (ej: 'la segunda', 'L1' o el nombre)."

    qty = max(1, _safe_int(qty, 1))

    # (1) ordinal
    idx = normalize_ordinal_to_index(ref)
    if idx is not None:
        candidates = STATE.get("last_candidates", []) or []
        if 0 <= idx < len(candidates):
            chosen = candidates[idx]
            cart_add_item(chosen, qty=qty)
            STATE["selected_product_id"] = chosen["id"]
            STATE["stage"] = "decide"
            return f"✅ Agregado: {chosen['nombre']} x{qty}\n\n{cart_to_text()}"
        return "No encontré esa opción. Dime 1, 2 o 3."

    # (2) resolver por id o embedding
    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    try:
        with driver.session() as session:
            prod = None
            looks_like_id = len(ref) <= 6 and ref[:1].isalpha()
            if looks_like_id:
                prod = session.run(
                    "MATCH (p:Producto {id:$id}) RETURN p.id AS id, p.nombre AS nombre, p.precio AS precio LIMIT 1",
                    id=ref.upper()
                ).single()
            if not prod:
                v = embedder.encode(ref).tolist()
                prod = session.run("""
                    CALL db.index.vector.queryNodes('productos_embeddings', 1, $vector)
                    YIELD node AS p, score
                    WHERE score > 0.6
                    RETURN p.id AS id, p.nombre AS nombre, p.precio AS precio
                """, vector=v).single()

            if not prod:
                return "No pude identificar ese producto para agregar al carrito."

            item = {"id": prod["id"], "nombre": prod["nombre"], "precio": prod["precio"]}
            cart_add_item(item, qty=qty)
            STATE["selected_product_id"] = item["id"]
            STATE["stage"] = "decide"

            return f"✅ Agregado: {item['nombre']} x{qty}\n\n{cart_to_text()}"
    finally:
        driver.close()


@tool
def ver_carrito() -> str:
    """Muestra el carrito actual."""
    logger.info("🛠️ [TOOL] ver_carrito")
    return cart_to_text()


@tool
def vaciar_carrito() -> str:
    """Vacía el carrito (solo si el usuario lo pidió explícitamente)."""
    logger.info("🛠️ [TOOL] vaciar_carrito")
    cart_clear()
    return "🧹 Listo. Carrito vaciado."


@tool
def remover_del_carrito(items_ref: str) -> str:
    """
    Remueve UNO o VARIOS ítems del carrito.
    Soporta:
    - Por nombre/id: "logitech, razer y dell" / "L2"
    - Por índice: "1 y 3" / "item 2" / "items 1,3"
    """
    logger.info(f"🛠️ [TOOL] remover_del_carrito | items_ref='{items_ref}'")
    if not STATE.get("cart_items"):
        return "Tu carrito ya está vacío."

    raw = (items_ref or "").strip().lower()
    if not raw:
        return "¿Qué quieres quitar del carrito? (ej: 'quita el 1', 'quita logitech')."

    # (A) índices
    idxs = parse_cart_indexes(raw)
    if idxs:
        removed_items = cart_remove_by_indexes(idxs)
        msg = ""
        if removed_items:
            msg += "✅ Quité del carrito: " + ", ".join(f"{it.get('nombre')} [{it.get('id')}]" for it in removed_items) + ".\n"
        else:
            msg += "⚠️ No pude quitar esos índices.\n"
        msg += "\n" + cart_to_text()
        return msg

    # (B) nombres/ids
    raw = raw.replace(" y ", ",")
    parts = [p.strip() for p in raw.split(",") if p.strip()]

    removed_any = False
    removed_list = []
    not_found = []

    for part in parts:
        removed = cart_remove_by_name_or_id(part)
        if removed:
            removed_any = True
            removed_list.append(part)
        else:
            not_found.append(part)

    msg = ""
    if removed_any:
        msg += f"✅ Quité del carrito: {', '.join(removed_list)}.\n"
    if not_found:
        msg += f"⚠️ No encontré en el carrito: {', '.join(not_found)}.\n"
    msg += "\n" + cart_to_text()
    return msg


@tool
def verificar_stock(producto_ref: str = "", tienda: str = "") -> str:
    """
    Devuelve stock por tienda.
    Si producto_ref está vacío, usa selected_product_id del estado.
    """
    producto_ref = (producto_ref or "").strip()
    if not producto_ref:
        if STATE.get("selected_product_id"):
            producto_ref = str(STATE["selected_product_id"])
        else:
            return "¿De cuál producto? (puedes decir 'la segunda' o escribir el nombre)."

    logger.info(f"🛠️ [TOOL] verificar_stock | producto_ref='{producto_ref}' tienda='{tienda}'")
    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

    # Resolver producto por ID exacto si parece ID, si no por embeddings
    is_id = len(producto_ref.strip()) <= 6 and producto_ref.strip().upper()[0].isalpha()

    try:
        with driver.session() as session:
            prod = None
            if is_id:
                prod = session.run(
                    "MATCH (p:Producto {id:$id}) RETURN p.id AS id, p.nombre AS nombre LIMIT 1",
                    id=producto_ref.strip().upper()
                ).single()
            if not prod:
                v = embedder.encode(producto_ref).tolist()
                prod = session.run("""
                    CALL db.index.vector.queryNodes('productos_embeddings', 1, $vector)
                    YIELD node AS p, score
                    WHERE score > 0.6
                    RETURN p.id AS id, p.nombre AS nombre
                """, vector=v).single()

            if not prod:
                return "No pude identificar el producto para revisar stock."

            pid = prod["id"]
            pname = prod["nombre"]
            STATE["selected_product_id"] = pid
            STATE["stage"] = "buy"

            if tienda.strip():
                rows = session.run("""
                    MATCH (t:Tienda {nombre:$tienda})-[s:TIENE_STOCK]->(p:Producto {id:$pid})
                    RETURN t.nombre AS tienda, s.cantidad AS cantidad
                """, tienda=tienda.strip(), pid=pid)
            else:
                rows = session.run("""
                    MATCH (t:Tienda)-[s:TIENE_STOCK]->(p:Producto {id:$pid})
                    RETURN t.nombre AS tienda, s.cantidad AS cantidad
                    ORDER BY cantidad DESC
                """, pid=pid)

            data = [dict(r) for r in rows]
            if not data:
                return f"❌ No hay stock registrado para {pname}."

            txt = f"✅ Stock para {pname} [{pid}]:\n"
            for r in data:
                txt += f"- {r['tienda']}: {r['cantidad']} unid.\n"
            return txt
    finally:
        driver.close()


@tool
def verificar_stock_carrito(tienda: str = "") -> str:
    """
    Revisa stock para TODO el carrito.
    Si NO se especifica tienda, elige la mejor para el primer ítem y la guarda en selected_store.
    """
    logger.info(f"🛠️ [TOOL] verificar_stock_carrito | tienda='{tienda}'")
    cart = STATE.get("cart_items", []) or []
    if not cart:
        return "Tu carrito está vacío. Dime qué productos quieres comprar."

    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    lines = []
    best_store_to_save = None

    try:
        with driver.session() as session:
            for c in cart:
                pid = c.get("id")
                pname = c.get("nombre", pid)
                qty = int(c.get("qty", 1))

                if tienda.strip():
                    rows = session.run("""
                        MATCH (t:Tienda {nombre:$tienda})-[s:TIENE_STOCK]->(p:Producto {id:$pid})
                        RETURN t.nombre AS tienda, s.cantidad AS cantidad
                    """, tienda=tienda.strip(), pid=pid)
                else:
                    rows = session.run("""
                        MATCH (t:Tienda)-[s:TIENE_STOCK]->(p:Producto {id:$pid})
                        RETURN t.nombre AS tienda, s.cantidad AS cantidad
                        ORDER BY cantidad DESC
                    """, pid=pid)

                data = [dict(r) for r in rows]
                if not data:
                    lines.append(f"❌ {pname} [{pid}] x{qty}: sin stock.")
                    continue

                best = data[0]
                ok = int(best["cantidad"]) >= qty
                lines.append(
                    f"{'✅' if ok else '⚠️'} {pname} [{pid}] x{qty}: mejor -> {best['tienda']} ({best['cantidad']} unid.)"
                )

                if not tienda.strip() and best_store_to_save is None:
                    best_store_to_save = best["tienda"]

        STATE["stage"] = "buy"
        if best_store_to_save:
            STATE["selected_store"] = best_store_to_save

        return "Stock del carrito:\n" + "\n".join(lines) + "\n\n" + cart_to_text()
    finally:
        driver.close()


@tool
def obtener_contacto_tienda(nombre_tienda: str = "") -> str:
    """Devuelve teléfono/WhatsApp/horario/dirección de la tienda. Si vacío, usa selected_store."""
    nombre_tienda = (nombre_tienda or "").strip()
    if not nombre_tienda:
        nombre_tienda = STATE.get("selected_store") or ""

    logger.info(f"🛠️ [TOOL] obtener_contacto_tienda | tienda='{nombre_tienda}'")

    if not nombre_tienda:
        return "¿De qué tienda? Opciones: Tienda Central, Sucursal Norte, Venta Online."

    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    try:
        with driver.session() as session:
            row = session.run("""
                MATCH (t:Tienda)
                WHERE toLower(t.nombre) CONTAINS toLower($name)
                RETURN t.nombre AS nombre, t.canal AS canal, t.telefono AS telefono,
                       t.whatsapp AS whatsapp, t.direccion AS direccion, t.horario AS horario
                LIMIT 1
            """, name=nombre_tienda).single()

        if not row:
            return "No encontré esa tienda. Opciones: Tienda Central, Sucursal Norte, Venta Online."

        STATE["selected_store"] = row["nombre"]
        STATE["stage"] = "contact"

        def safe(v):
            return v if v not in (None, "") else "N/A"

        return (
            f"📍 {row['nombre']} ({safe(row.get('canal'))})\n"
            f"☎️ Tel: {safe(row.get('telefono'))}\n"
            f"💬 WhatsApp: {safe(row.get('whatsapp'))}\n"
            f"🕒 Horario: {safe(row.get('horario'))}\n"
            f"📌 Dirección: {safe(row.get('direccion'))}\n"
        )
    finally:
        driver.close()


@tool
def finalizar_compra(tienda: str = "") -> str:
    """
    Finaliza compra (simulada):
    - Valida carrito
    - Elige tienda (si no viene, usa selected_store o la mejor por stock del primer ítem)
    - Verifica stock suficiente en esa tienda para todos los items
    - Devuelve "✅ Compra realizada" + tienda + contacto + lista de productos
    - Vacía el carrito
    """
    tienda = (tienda or "").strip()
    logger.info(f"🛠️ [TOOL] finalizar_compra | tienda='{tienda}'")

    cart = STATE.get("cart_items", []) or []
    if not cart:
        return "Tu carrito está vacío. Dime qué productos quieres comprar."

    best_store = tienda or STATE.get("selected_store")

    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    try:
        with driver.session() as session:
            # Si no hay tienda, elegimos la mejor para el primer item por stock
            if not best_store:
                first = cart[0]
                pid = first.get("id")
                row = session.run("""
                    MATCH (t:Tienda)-[s:TIENE_STOCK]->(p:Producto {id:$pid})
                    RETURN t.nombre AS tienda, s.cantidad AS cantidad
                    ORDER BY cantidad DESC
                    LIMIT 1
                """, pid=pid).single()
                if row:
                    best_store = row["tienda"]

            if not best_store:
                return "No pude determinar una tienda. ¿Prefieres Tienda Central, Sucursal Norte o Venta Online?"

            # Verificar stock de todo el carrito en esa tienda
            faltantes = []
            for c in cart:
                pid = c.get("id")
                pname = c.get("nombre", pid)
                qty = int(c.get("qty", 1))

                row = session.run("""
                    MATCH (t:Tienda {nombre:$tienda})-[s:TIENE_STOCK]->(p:Producto {id:$pid})
                    RETURN s.cantidad AS cantidad
                    LIMIT 1
                """, tienda=best_store, pid=pid).single()

                if not row or int(row["cantidad"]) < qty:
                    faltantes.append(f"- {pname} [{pid}] x{qty}")

            if faltantes:
                return (
                    f"⚠️ No hay stock suficiente en **{best_store}** para:\n"
                    + "\n".join(faltantes)
                    + "\n\nDime si quieres intentar otra tienda (Central/Norte/Online)."
                )

            # Contacto
            t = session.run("""
                MATCH (t:Tienda {nombre:$name})
                RETURN t.nombre AS nombre, t.canal AS canal, t.telefono AS telefono,
                       t.whatsapp AS whatsapp, t.direccion AS direccion, t.horario AS horario
                LIMIT 1
            """, name=best_store).single()

            if not t:
                return f"Encontré stock, pero no encontré datos de contacto para la tienda '{best_store}'."

        def safe(v):
            return v if v not in (None, "") else "N/A"

        ticket = cart_to_text()

        # Vaciar carrito + estado
        STATE["selected_store"] = best_store
        STATE["stage"] = "done"
        cart_clear()

        return (
            "✅ **Compra realizada**\n\n"
            f"📍 **Acércate a:** {safe(t['nombre'])} ({safe(t.get('canal'))})\n"
            f"📌 Dirección: {safe(t.get('direccion'))}\n"
            f"🕒 Horario: {safe(t.get('horario'))}\n"
            f"☎️ Tel: {safe(t.get('telefono'))}\n"
            f"💬 WhatsApp: {safe(t.get('whatsapp'))}\n\n"
            "🧾 **Productos comprados:**\n"
            f"{ticket}"
        )
    finally:
        driver.close()


TOOLS = [
    buscar_productos,
    agregar_al_carrito,
    remover_del_carrito,
    ver_carrito,
    vaciar_carrito,
    verificar_stock,
    verificar_stock_carrito,
    obtener_contacto_tienda,
    finalizar_compra,
]


# =========================================================
# 4. ROUTER (FASE 2) - actualizado
# =========================================================
ROUTER_LABELS = [
    "browse_products",
    "add_to_cart",
    "remove_from_cart",
    "view_cart",
    "purchase_or_stock",
    "finalize_purchase",
    "store_contact",
    "respuesta_directa",
]
ROUTER_EMBEDS = embedder.encode(ROUTER_LABELS, normalize_embeddings=True)

def cosine_sim(a, b) -> float:
    return float(sum(x * y for x, y in zip(a, b)))

def seleccionar_funcion(query_vec_norm: List[float]):
    sims = [cosine_sim(query_vec_norm, ROUTER_EMBEDS[i]) for i in range(len(ROUTER_LABELS))]
    best_idx = max(range(len(sims)), key=lambda i: sims[i])
    return ROUTER_LABELS[best_idx], float(sims[best_idx]), sims


# =========================================================
# 5. PLANNER (FASE 3) - actualizado
# =========================================================
PLANNER_SYSTEM = """
Eres un PLANNER. Devuelve SOLO JSON válido.

Formato:
{"steps":[{"tool":"buscar_productos","args":{"query":"..."}}]}

REGLAS IMPORTANTES:
- NUNCA inventes productos en el carrito.
- SOLO agrega al carrito si el usuario lo pide explícitamente.

Reglas:
- Explorar ("busco", "quiero ver", "recomiéndame", "tienes laptops") => buscar_productos(query).

- Agregar al carrito ("quiero X", "añade X", "me llevo X"):
  => agregar_al_carrito(producto_ref="X", qty=1) (si hay dos productos "X y Y", genera 2 steps).

- Quitar del carrito ("quita X, Y y Z" o "quita 1 y 3"):
  => remover_del_carrito(items_ref="...") en UN step.
  Luego agrega ver_carrito().

- Ver carrito ("mi carrito", "qué tengo", "ver carrito") => ver_carrito().

- Vaciar carrito SOLO si dice "vacía el carrito", "borra todo", "elimina todo" => vaciar_carrito().

- Stock / compra:
  - Si dice "quiero comprar" => verificar_stock_carrito(tienda="") si hay carrito.
  - Si pide stock de un producto => verificar_stock(producto_ref="...", tienda="").

- Dónde comprar ("dónde", "muéstrame dónde", "en qué tienda", "a dónde voy", "dónde lo consigo"):
  Si hay carrito:
    1) verificar_stock_carrito(tienda="")
    2) obtener_contacto_tienda(nombre_tienda="")  # usa selected_store
  Si no hay carrito:
    1) verificar_stock(producto_ref="", tienda="")
    2) obtener_contacto_tienda(nombre_tienda="")

- Proceder a compra / finalizar ("proceder a compra", "finalizar compra", "comprar ya", "listo comprar"):
  Si hay carrito => finalizar_compra(tienda="")

- Contacto explícito ("número", "whatsapp", "llamar", "dirección", "horario", "contacto"):
  => obtener_contacto_tienda(nombre_tienda="...") (si no da tienda, usa "")

- Si no necesitas herramientas => {"steps": []}

Tools válidas:
- buscar_productos(query)
- agregar_al_carrito(producto_ref, qty)
- remover_del_carrito(items_ref)
- ver_carrito()
- vaciar_carrito()
- verificar_stock(producto_ref, tienda opcional)
- verificar_stock_carrito(tienda opcional)
- obtener_contacto_tienda(nombre_tienda opcional)
- finalizar_compra(tienda opcional)
"""

def planificar(query_text: str, router_label: str) -> Dict[str, Any]:
    prompt = (
        f"Usuario: {query_text}\n"
        f"Router: {router_label}\n"
        f"Estado: {json.dumps(STATE, ensure_ascii=False)}\n"
        f"Devuelve el plan JSON."
    )
    msg = llm.invoke([SystemMessage(content=PLANNER_SYSTEM), HumanMessage(content=prompt)])
    raw = (msg.content or "").strip()
    try:
        plan = json.loads(raw)
        if not isinstance(plan, dict) or "steps" not in plan or not isinstance(plan["steps"], list):
            raise ValueError("Plan inválido.")
        return plan
    except Exception as e:
        logger.warning(f"⚠️ [PLANNER] JSON inválido => steps=[] | err={e} | raw={raw[:200]}")
        return {"steps": []}


# =========================================================
# 6. EXECUTOR (FASE 4)
# =========================================================
def ejecutar_plan(plan: Dict[str, Any]) -> List[Dict[str, Any]]:
    results = []
    for i, step in enumerate(plan.get("steps", []), start=1):
        tool_name = step.get("tool")
        args = step.get("args", {}) or {}
        logger.info(f"⚙️ [EXEC] Step {i} | tool={tool_name} | args={args}")

        if tool_name == "buscar_productos":
            out = buscar_productos.invoke(args)
        elif tool_name == "agregar_al_carrito":
            out = agregar_al_carrito.invoke(args)
        elif tool_name == "remover_del_carrito":
            out = remover_del_carrito.invoke(args)
        elif tool_name == "ver_carrito":
            out = ver_carrito.invoke(args)
        elif tool_name == "vaciar_carrito":
            out = vaciar_carrito.invoke(args)
        elif tool_name == "verificar_stock":
            out = verificar_stock.invoke(args)
        elif tool_name == "verificar_stock_carrito":
            out = verificar_stock_carrito.invoke(args)
        elif tool_name == "obtener_contacto_tienda":
            out = obtener_contacto_tienda.invoke(args)
        elif tool_name == "finalizar_compra":
            out = finalizar_compra.invoke(args)
        else:
            out = f"Tool desconocida: {tool_name}"

        results.append({"tool": tool_name, "args": args, "output": out})
    return results


# =========================================================
# 7. RESPONDER (FASE 5) - actualizado
# =========================================================
RESPONDER_SYSTEM = """
Eres un vendedor conversacional.

Reglas:
- Si el usuario explora, muestra opciones y pregunta 1 cosa útil (presupuesto/uso).
- No muestres stock salvo que el usuario lo pida o diga que quiere comprar.
- Si el usuario agrega al carrito, confirma carrito y pregunta: "¿quieres ver stock o proceder a compra?"
- Si el usuario pide quitar, muestra carrito actualizado.
- Si se ejecutó finalizar_compra, NO preguntes más: solo confirma compra y da tienda + productos.
- Si el usuario pide "dónde comprar", entrega tienda + contacto sin volver a preguntar.
- Si falta dato clave, pregunta breve.
"""

def redactar_respuesta(query_text: str, tool_results: List[Dict[str, Any]]) -> str:
    contexto = ""
    for r in tool_results:
        contexto += f"[TOOL={r['tool']} ARGS={r['args']}]\n{r['output']}\n\n"

    msg = llm.invoke([
        SystemMessage(content=RESPONDER_SYSTEM),
        HumanMessage(content=(
            f"Usuario:\n{query_text}\n\n"
            f"Estado:\n{json.dumps(STATE, ensure_ascii=False)}\n\n"
            f"Contexto:\n{contexto}"
        ))
    ])
    return msg.content


# =========================================================
# 8. PIPELINE 1→5
# =========================================================
def ejecutar_pipeline(query_text: str, trace_id: str) -> Dict[str, Any]:
    logger.info(f"🧩 [FASE 1] ({trace_id}) Query -> Embedding")
    q_vec = embedder.encode(query_text, normalize_embeddings=True).tolist()

    logger.info(f"🧭 [FASE 2] ({trace_id}) Function Selection")
    router_label, router_score, sims = seleccionar_funcion(q_vec)
    logger.info(f"🧭 ({trace_id}) Router='{router_label}' score={router_score:.4f}")

    logger.info(f"🗺️ [FASE 3] ({trace_id}) Planner")
    plan = planificar(query_text, router_label)
    logger.info(f"🗺️ ({trace_id}) Plan={plan}")

    logger.info(f"⚙️ [FASE 4] ({trace_id}) Ejecutando plan")
    tool_results = ejecutar_plan(plan)

    logger.info(f"💬 [FASE 5] ({trace_id}) Respuesta final")
    response = redactar_respuesta(query_text, tool_results)

    return {
        "response": response,
        "router": {"label": router_label, "score": router_score, "sims": sims},
        "plan": plan,
        "tool_results": tool_results,
        "state": dict(STATE),
    }


# =========================================================
# 9. LOOP + MLFLOW (Notebook/Terminal)
# =========================================================
def prueba_interactiva():
    trace_id = str(uuid.uuid4())[:8]
    pregunta = input("\n👤 Usuario: ")

    with mlflow.start_run() as run:
        mlflow.log_param("trace_id", trace_id)
        mlflow.log_param("query", pregunta)

        result = ejecutar_pipeline(pregunta, trace_id)
        print(f"\n🤖 AGENTE: {result['response']}\n")

        mlflow.log_text(json.dumps(result["plan"], ensure_ascii=False, indent=2), "plan.json")
        mlflow.log_text(result["response"], "respuesta_agente.txt")
        mlflow.log_text(json.dumps(result["state"], ensure_ascii=False, indent=2), "state.json")

        ok = input("¿Fue útil? (s/n): ").strip().lower()
        score = 1 if ok == "s" else 0
        mlflow.log_metric("helpfulness", score)

        if score == 0:
            comentario = input("¿Qué estuvo mal?: ").strip()
            mlflow.log_text(comentario, "feedback_negativo.txt")


if __name__ == "__main__":
    while True:
        prueba_interactiva()
        if input("\n¿Otra? (s/n): ").lower() != "s":
            break


23:56:20 | WARNING | NOTEBOOK_AGENTE | ⚠️ Detectado entorno local: Cambiando host 'neo4j' por 'localhost'
2026/02/02 23:56:21 INFO mlflow.tracking.fluent: Experiment with name 'Agente_Conversacional_Planner' does not exist. Creating a new experiment.
23:56:21 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cpu
23:56:21 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: all-MiniLM-L6-v2
23:56:22 | INFO | httpx | HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
23:56:22 | INFO | httpx | HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json "HTTP/1.1 200 OK"
23:56:22 | INFO | httpx | HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Tem


🤖 AGENTE: ¡Genial que estés buscando una laptop! Aquí tienes algunas opciones:

1. **MacBook Air M2** - $1200
   - Laptop ligera con chip M2 de Apple, pantalla de 13 pulgadas.
   
2. **Dell XPS 13** - $1100
   - Ultrabook con Windows y pantalla InfinityEdge.
   
3. **Lenovo ThinkPad X1** - $1400
   - Laptop empresarial ultrarresistente, hecha de fibra de carbono.

Para ayudarte mejor, ¿tienes un presupuesto en mente o un uso específico para la laptop?

🏃 View run kindly-boar-76 at: http://localhost:5000/#/experiments/2/runs/1134daa19a0342f2b9614a9cbc51b8f9
🧪 View experiment at: http://localhost:5000/#/experiments/2
